In [ ]:
# SISO 5G gNB-UE Simulation using AIRSTRAN D 2200

# Import or install Sionna
try:
    import sionna.rt
except ImportError as e:
    import os
    os.system("pip install sionna-rt")
    import sionna.rt

# Other imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import mitsuba as mi
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", message="invalid value encountered in multiply")
warnings.filterwarnings("ignore", category=UserWarning, module="jupyter_client")

# Import relevant components from Sionna RT
from sionna.rt import load_scene, Transmitter, Receiver, Camera, PathSolver, \
                      AntennaArray, PlanarArray, SceneObject, ITURadioMaterial
from sionna.rt.antenna_pattern import antenna_pattern_registry

# Load empty scene
scene = load_scene("/home/tingjunlab/Development/geo2sigmap/scenes/Duke_Perkins/scene.xml")

In [ ]:
# Install plyfile if not available
try:
    from plyfile import PlyData
except ImportError:
    import os
    os.system("pip install plyfile")
    from plyfile import PlyData

In [ ]:
# ============================================
# SISO Configuration: gNB to UE
# ============================================

# Set the operating frequency (n48 band for 5G)
scene.frequency = 3.7e9  # 3.7 GHz

# Define UE position
ue_position = [10.0, 0.0, 0.0]   # UE position (x, y, z in meters)

# ============================================
# Antenna Configuration
# ============================================

# gNB antenna: 3GPP TR 38.901 pattern (AIRSTRAN D 2200)
gnb_pattern_factory = antenna_pattern_registry.get("tr38901")
gnb_pattern = gnb_pattern_factory(polarization="V")

# UE antenna: Isotropic pattern (typical for mobile devices)
ue_pattern_factory = antenna_pattern_registry.get("iso")
ue_pattern = ue_pattern_factory(polarization="V")

# SISO: Single antenna element at origin [0, 0, 0] for both TX and RX
single_element = np.array([[0.0, 0.0, 0.0]])  # Shape: (1, 3)

# Configure antenna arrays
scene.tx_array = AntennaArray(
    antenna_pattern=gnb_pattern,
    normalized_positions=single_element.T  # Shape: (3, 1)
)

scene.rx_array = AntennaArray(
    antenna_pattern=ue_pattern,
    normalized_positions=single_element.T  # Shape: (3, 1)
)

# ============================================
# Add Receiver to Scene
# ============================================

# Create UE receiver
rx = Receiver(name="ue", position=ue_position, display_radius=0.03)
scene.add(rx)

# ============================================
# Configure Propagation Environment
# ============================================

# Disable scattering for basic simulation
for radio_material in scene.radio_materials.values():
    radio_material.scattering_coefficient = 0.0

In [ ]:
# ============================================
# Building Selection and gNB Placement Helper
# ============================================

import xml.etree.ElementTree as ET
from pathlib import Path

def parse_scene_buildings(scene_xml_path):
    """
    Parse scene.xml to extract building information

    Parameters:
    -----------
    scene_xml_path : str
        Path to the scene.xml file

    Returns:
    --------
    buildings : dict
        Dictionary mapping building_id to rooftop mesh path
    """
    tree = ET.parse(scene_xml_path)
    root = tree.getroot()

    buildings = {}

    # Find all rooftop mesh shapes
    for shape in root.findall(".//shape[@type='ply']"):
        shape_id = shape.get('id', '')
        if 'rooftop' in shape_id:
            # Extract building number from id like "mesh-building_0_rooftop"
            parts = shape_id.split('_')
            if len(parts) >= 3 and parts[0] == 'mesh-building':
                building_id = int(parts[1])
                filename = shape.find("string[@name='filename']").get('value')
                buildings[building_id] = filename

    return buildings

def load_rooftop_mesh(mesh_path):
    """
    Load rooftop PLY mesh and extract vertices

    Parameters:
    -----------
    mesh_path : str
        Path to the rooftop PLY file

    Returns:
    --------
    vertices : numpy array
        Nx3 array of vertex coordinates
    z_height : float
        Z-coordinate (height) of the rooftop
    """
    from plyfile import PlyData

    plydata = PlyData.read(mesh_path)
    vertices = plydata['vertex']

    # Extract x, y, z coordinates
    x = np.array(vertices['x'])
    y = np.array(vertices['y'])
    z = np.array(vertices['z'])

    # Stack into Nx3 array
    coords = np.column_stack([x, y, z])

    # Get the rooftop height (max z value)
    z_height = np.max(z)

    return coords, z_height

def list_buildings(scene_xml_path):
    """
    List all available buildings in the scene

    Parameters:
    -----------
    scene_xml_path : str
        Path to the scene.xml file
    """
    buildings = parse_scene_buildings(scene_xml_path)

    print(f"Available buildings in scene: {len(buildings)} total")
    print("=" * 80)

    scene_dir = Path(scene_xml_path).parent
    building_info = {}

    for building_id in sorted(buildings.keys()):
        mesh_file = scene_dir / buildings[building_id]

        if mesh_file.exists():
            try:
                coords, z_height = load_rooftop_mesh(str(mesh_file))

                # Calculate bounding box
                x_min, y_min = coords[:, 0].min(), coords[:, 1].min()
                x_max, y_max = coords[:, 0].max(), coords[:, 1].max()
                x_center = (x_min + x_max) / 2
                y_center = (y_min + y_max) / 2

                building_info[building_id] = {
                    'z_height': z_height,
                    'x_range': (x_min, x_max),
                    'y_range': (y_min, y_max),
                    'center': (x_center, y_center),
                    'vertices': coords
                }

                print(f"\nBuilding {building_id}:")
                print(f"  Rooftop height: {z_height:.2f} m")
                print(f"  X range: [{x_min:.2f}, {x_max:.2f}] m (center: {x_center:.2f})")
                print(f"  Y range: [{y_min:.2f}, {y_max:.2f}] m (center: {y_center:.2f})")
                print(f"  Vertices: {len(coords)}")

            except Exception as e:
                print(f"\nBuilding {building_id}: Error loading mesh - {e}")
        else:
            print(f"\nBuilding {building_id}: Mesh file not found")

    print("=" * 80)
    return building_info

def place_tx_on_building(building_id, x_pos, y_pos, scene_xml_path, offset_height=2.0):
    """
    Place transmitter on a specific building at given X/Y coordinates

    Parameters:
    -----------
    building_id : int
        Building number (0-49 for Duke_Perkins scene)
    x_pos : float
        X coordinate on the rooftop (in meters)
    y_pos : float
        Y coordinate on the rooftop (in meters)
    scene_xml_path : str
        Path to the scene.xml file
    offset_height : float
        Height above the rooftop surface (default 2.0 meters)

    Returns:
    --------
    position : list
        [x, y, z] position for the transmitter (as float32)
    """
    buildings = parse_scene_buildings(scene_xml_path)

    if building_id not in buildings:
        print(f"Error: Building {building_id} not found in scene")
        print(f"Available building IDs: {sorted(buildings.keys())}")
        return None

    # Load the rooftop mesh
    scene_dir = Path(scene_xml_path).parent
    mesh_file = scene_dir / buildings[building_id]

    try:
        coords, z_height = load_rooftop_mesh(str(mesh_file))

        # Calculate valid range
        x_min, x_max = coords[:, 0].min(), coords[:, 0].max()
        y_min, y_max = coords[:, 1].min(), coords[:, 1].max()

        # Validate coordinates are within rooftop bounds
        if not (x_min <= x_pos <= x_max):
            print(f"Warning: X position {x_pos:.2f} is outside rooftop range [{x_min:.2f}, {x_max:.2f}]")
            print(f"  Clamping to valid range...")
            x_pos = np.clip(x_pos, x_min, x_max)

        if not (y_min <= y_pos <= y_max):
            print(f"Warning: Y position {y_pos:.2f} is outside rooftop range [{y_min:.2f}, {y_max:.2f}]")
            print(f"  Clamping to valid range...")
            y_pos = np.clip(y_pos, y_min, y_max)

        # Calculate final position - convert to float32 for Mitsuba compatibility
        tx_z = z_height + offset_height
        position = [float(x_pos), float(y_pos), float(tx_z)]

        print(f"\nTransmitter Placement:")
        print(f"  Building ID: {building_id}")
        print(f"  Rooftop height: {z_height:.2f} m")
        print(f"  TX position: ({position[0]:.2f}, {position[1]:.2f}, {position[2]:.2f})")
        print(f"  Height above rooftop: {offset_height:.2f} m")
        print(f"  Rooftop bounds: X=[{x_min:.2f}, {x_max:.2f}], Y=[{y_min:.2f}, {y_max:.2f}]")

        return position

    except Exception as e:
        print(f"Error loading building mesh: {e}")
        return None

# Define scene path
scene_xml_path = "/home/tingjunlab/Development/geo2sigmap/scenes/Duke_Perkins/scene.xml"

# List all buildings in the Duke Perkins scene
building_info = list_buildings(scene_xml_path)

In [ ]:
# ============================================
# Place gNB on a Specific Building
# ============================================

# STEP 1: Choose a building from the list above
# Buildings are numbered 0-49 in the Duke_Perkins scene

scene.tx_array.antenna_pattern.show();

selected_building_id = 5  # Change this to your desired building number

# STEP 2: Choose X/Y position on the rooftop
# Use the building info from the list above to see valid ranges
# You can use the center, or specify custom coordinates

# Option A: Use building center
building = building_info[selected_building_id]
x_position = building['center'][0]
y_position = building['center'][1]

# Option B: Custom X/Y coordinates (uncomment to use)
# x_position = 100.0  # Your desired X coordinate
# y_position = 50.0   # Your desired Y coordinate

# STEP 3: Place the gNB
gnb_position = place_tx_on_building(
    building_id=selected_building_id,
    x_pos=x_position,
    y_pos=y_position,
    scene_xml_path=scene_xml_path,
    offset_height=5.0  # Height above rooftop (default 3 meters)
)

# STEP 4: Add the transmitter to the scene
if gnb_position is not None:
    # Create transmitter at the building location
    tx = Transmitter(name="gnb", position=gnb_position, display_radius=0.5)
    scene.add(tx)

    # Point antenna toward UE
    tx.look_at(ue_position)

    print(f"\nSuccess! gNB placed on building {selected_building_id}")
    print(f"Position: {gnb_position}")
    
    # ============================================
    # Compute Propagation Paths
    # ============================================
    
    # Instantiate path solver
    p_solver = PathSolver()
    
    # Compute propagation paths
    paths = p_solver(
        scene=scene,
        max_depth=5,
        max_num_paths_per_src=5000,
        los=True,
        specular_reflection=True,
        diffuse_reflection=False,
        refraction=False,
        synthetic_array=False,
        seed=41,
    )
    
    # ============================================
    # Visualize Scene
    # ============================================
    
    # Setup camera
    cam = Camera(position=(100.0, 100.0, 50.0))
    cam.look_at(gnb_position)
    
    # Preview the scene with propagation paths
    scene.preview(
        paths=paths,
        resolution=[1000, 1000],
        clip_at=50,
        show_orientations=True
    )
else:
    print("\nFailed to place gNB. Check the building ID and coordinates.")

In [ ]:
from boresight_pathsolver import (
    optimize_boresight_pathsolver,
    create_target_radiomap
)
import matplotlib.pyplot as plt

# Map configuration (same as before)
map_config = {
    'center': [gnb_position[0], gnb_position[1], 0],
    'size': [300, 300],
    'cell_size': (10, 10),
    'ground_height': 0.0,
}

angle_map = [{'angle_start': 0, 'angle_end': 120, 'power_dbm': -70, 'relative_power': 'high'},
            {'angle_start': 120, 'angle_end': 360, 'power_dbm': -100, 'relative_power': 'low'}]

# CHANGE: Use path_loss_sector instead of angular_sectors
target_map = create_target_radiomap(
    map_config,
    target_type='angular_sectors',
    angular_sectors=angle_map,
    tx_position=gnb_position,
    auto_scale_power=True,
    #sector_angle=45,        
    #sector_width=120,      
    #frequency_GHz=3.5,
    #path_loss_exponent=2.5
)

# Calculate the "center" look-at position. The look_at position should be level with the BS height.
center_x = map_config['center'][0] + 50.0
center_y = map_config['center'][1] + 50.0
center_z = map_config['center'][2]

# Rest is the same
best_boresight, loss_hist, bore_hist, grad_hist = optimize_boresight_pathsolver(
    scene=scene,
    tx_name="gnb",
    map_config=map_config,
    building_info=building_info,
    target_map=target_map,
    initial_boresight=[10.0, 10.0, 10.0],
    num_sample_points=100,
    learning_rate=.2,      
    num_iterations=200,     
    loss_type='mse',
    verbose=True,
    seed=33,
    # Enable frame saving
    save_radiomap_frames=True,
    frame_save_interval=10,
    output_dir="/home/tingjunlab/Development/optimize_tx/optimization_gif"
)

print(f"Optimized boresight: ({best_boresight[0]:.1f}, {best_boresight[1]:.1f})")

In [ ]:
from boresight_pathsolver import create_optimization_gif

# Create GIF from saved frames
create_optimization_gif(
    frame_dir="/home/tingjunlab/Development/optimize_tx/optimization_gif",
    output_path="boresight_optimization.gif",
    duration=300,  # 300ms per frame
    loop=0  # Loop forever
)

In [ ]:
from sionna.rt import RadioMapSolver
from diagnostic_utils import analyze_coverage, print_analysis, visualize_results
import diagnostic_utils, optimizer_diagnostics
from optimizer_diagnostics import diagnose_optimization, print_diagnostics, plot_diagnostics

# Apply the optimized boresight
#m_best_boresight = mi.Point3f(best_mse[0], best_mse[1], best_mse[2])
tx.look_at(mi.Point3f(float(best_boresight[0]), float(best_boresight[1]), float(best_boresight[2])))
#stx.position = mi.Point3f(float(best_tx[0]), float(best_tx[1]), float(building["z_height"] + 20.0))

print("="*70)
print("RADIO MAP VISUALIZATION")
print("="*70)
print(f"Optimized boresight: ({best_boresight[0]:.1f}, {best_boresight[1]:.1f}, {best_boresight[2]:.1f})")
print(f"TX location: {tx.position}")
print(f"Coverage center: {map_config['center']}")
print()

# Generate radio map EXACTLY matching the optimization configuration
rm_solver = RadioMapSolver()
rm = rm_solver(
    scene,
    max_depth=5,
    samples_per_tx=int(6e8),
    cell_size=map_config['cell_size'],  # ✓ USE map_config
    center=map_config['center'],        # ✓ USE map_config
    orientation=[0, 0, 0],
    size=map_config['size'],            # ✓ USE map_config
    los=True,
    specular_reflection=True,
    diffuse_reflection=True,
    refraction=False,
    stop_threshold=None,
)

print("Radio map configuration:")
print(f"  Center: {map_config['center']}")
print(f"  Size: {map_config['size']}")
print(f"  Cell size: {map_config['cell_size']}")
print()

# Extract signal levels
rss_watts = rm.rss.numpy()[0, :, :]
signal_strength_dBm = 10.0 * np.log10(rss_watts + 1e-30) + 30.0

# Automatic analysis based on target type and loss function
analysis = analyze_coverage(
    signal_strength_dBm,
    target_map,
    loss_type='mse',  # Change to 'cross_entropy' when using that loss
    map_config=map_config
)

# Print results
#success = print_analysis(analysis)

# Visualize
visualize_results(signal_strength_dBm, target_map, analysis)

# Diagnostics to adjust the optimization
diag = diagnose_optimization(loss_hist, bore_hist, grad_hist)
print_diagnostics(diag)
plot_diagnostics(loss_hist, bore_hist, grad_hist)

# Scene preview
scene.preview(radio_map=rm);